# Regional activity visualizations
This example shows how to create visualizations of iNaturalist activity in a given area using [Altair](https://altair-viz.github.io), with the following metrics:
* Number of observations
* Number of taxa observed
* Number of observers
* Number of identifiers

In [1]:
from datetime import datetime
from time import sleep

from dateutil.relativedelta import relativedelta
from typing import Any, BinaryIO, Dict, Iterable, List, Optional, Tuple

import altair as alt
import pandas as pd
from pyinaturalist.node_api import (
    get_observation_histogram,
    get_observation_species_counts,
    get_observation_observers,
    get_observation_identifiers,
)
from pyinaturalist.request_params import get_interval_ranges

PLACE_ID = 6
PLACE_NAME = 'Alaska'
YEAR = 2020
THROTTLING_DELAY = 1.0  # Time to wait in between subsequent requests

### Show observation counts by year

In [2]:
obs_by_year = get_observation_histogram(
    place_id=PLACE_ID,
    interval='year',
    d1='2008-01-01',
    d2=f'{YEAR}-12-31',
)
obs_by_year = pd.DataFrame([{'date': k, 'observations': v} for k, v in obs_by_year.items()])
alt.Chart(obs_by_year).mark_bar().encode(x='year(date):T', y='observations:Q')

alt.Chart(...)

### Show observation counts by month

In [3]:
obs_by_month = get_observation_histogram(
    place_id=PLACE_ID,
    interval='month',
    d1='2020-01-02',
    d2='2020-12-31',
)
observations_by_month = pd.DataFrame([{'metric': 'Observations', 'date': k, 'count': v} for k, v in obs_by_month.items()])
alt.Chart(observations_by_month).mark_bar().encode(x='month(date):T', y='count:Q')

alt.Chart(...)

### Making histograms for custom metrics
The API does not have a histogram endpoint for taxa observed, observers, or identifiers,
so we first need to determine our date ranges of interest, and then run one search per date range.

Here are some functions to simplify this.

In [4]:
def count_date_range_results(function, start_date, end_date):
    """Get the count of results for the given date range and search function"""
    # Running this search with per_page=0 will (quickly) return only a count of results, not complete results
    response = function(
        place_id=PLACE_ID,
        d1=start_date,
        d2=end_date,
        per_page=0,  
    )
    print(f'Total results for {start_date.strftime("%b")}: {response["total_results"]}')
    return response['total_results']
    if start_date.month != 12:
        sleep(THROTTLING_DELAY)


def get_monthly_counts(function, label):
    """Get the count of results per month for the given search function"""
    month_ranges = get_interval_ranges(datetime(YEAR, 1, 1), datetime(YEAR, 12, 31), 'monthly')
    counts_by_month = {
        start_date: count_date_range_results(function, start_date, end_date)
        for (start_date, end_date) in month_ranges
    }
    return pd.DataFrame([{'metric': label, 'date': k, 'count': v} for k, v in counts_by_month.items()])

### Show unique taxa observed per month

In [5]:
taxa_by_month = get_monthly_counts(get_observation_species_counts, 'Taxa')
alt.Chart(taxa_by_month).mark_bar().encode(x='month(date):T', y='count:Q')

Total results for Jan: 196
Total results for Feb: 179
Total results for Mar: 324
Total results for Apr: 800
Total results for May: 1372
Total results for Jun: 1502
Total results for Jul: 1728
Total results for Aug: 1605
Total results for Sep: 1268
Total results for Oct: 650
Total results for Nov: 413
Total results for Dec: 555


alt.Chart(...)

### Show number of observers per month

In [6]:
observers_by_month = get_monthly_counts(get_observation_observers, 'Observers')
alt.Chart(observers_by_month).mark_bar().encode(x='month(date):T', y='count:Q')

Total results for Jan: 44
Total results for Feb: 43
Total results for Mar: 79
Total results for Apr: 147
Total results for May: 376
Total results for Jun: 496
Total results for Jul: 572
Total results for Aug: 599
Total results for Sep: 439
Total results for Oct: 187
Total results for Nov: 91
Total results for Dec: 54


alt.Chart(...)

### Show number of identifiers per month

In [7]:
identifiers_by_month = get_monthly_counts(get_observation_identifiers, 'Identifiers')
alt.Chart(identifiers_by_month).mark_bar().encode(x='month(date):T', y='count:Q')

Total results for Jan: 141
Total results for Feb: 149
Total results for Mar: 188
Total results for Apr: 350
Total results for May: 624
Total results for Jun: 594
Total results for Jul: 664
Total results for Aug: 620
Total results for Sep: 497
Total results for Oct: 317
Total results for Nov: 216
Total results for Dec: 206


alt.Chart(...)

### Now, let's combine all four metrics into one chart

In [8]:
combined_results = observations_by_month.append([taxa_by_month, observers_by_month, identifiers_by_month])


alt.Chart(
    combined_results,
    title=f'iNaturalist activity in {PLACE_NAME} ({YEAR})',
    width=750,
    height=500,
).mark_line().encode(
    alt.X('month(date):T', axis=alt.Axis(title="Month")),
    alt.Y('count:Q', axis=alt.Axis(title="Count")),
    color='metric',
    strokeDash='metric',
).configure_axis(
    labelFontSize=15,
    titleFontSize=20,
)

alt.Chart(...)